# 3.1.2 形態學操作 (Morphological Operations)

**WBS 3.1.2**: 形態學操作 - 侵蝕、膨脹、開運算、閉運算、梯度、頂帽、黑帽變換

本模組涵蓋 OpenCV 形態學操作的完整介紹，從基本概念到實際應用。

<div style="page-break-after: always"></div>

# Module 9. 形態學

## 9-1: 形態學介紹
>影像處理中指的形態學，往往表示的是數學形態學。

>數學形態學（Mathematical morphology） 是一門建立在格論和拓撲學基礎之上的影像分析學科，是數學形態學影像處理的基本理論。其基本的運算包括：`二值侵蝕和膨脹、二值開閉運算、骨架抽取、極限侵蝕、擊中擊不中變換、形態學梯度、Top-hat變換、顆粒分析、流域變換、灰值侵蝕和膨脹、灰值開閉運算、灰值形態學梯度等`。

>簡單來講，形態學操作就是基於形狀的一系列影像處理操作。OpenCV為進行影像的形態學變換提供了快捷、方便的函式。最基本的形態學操作有二種，他們是：`侵蝕與膨脹` ( Erosion 與 Dilation )。

> 侵蝕與膨脹能實現多種多樣的功能，主要如下：

> * 消除噪聲
> * 分割 ( isolate ) 出獨立的影像元素，在影像中連線(join)相鄰的元素。
> * 尋找影像中的明顯的極大值區域或極小值區域
> * 求出影像的梯度

## 9-2: 什麼是形態學
> 形態學操作是根據影像形狀進行的簡單操作,一般情況下對`二值化影像`進行的操作。需要輸入兩個引數，一個是原始影像，第二個被稱為結構化元素或核，它是用來決定操作的性質的。<br>

> 兩個基本的形態學操作是侵蝕和膨脹。他們的變體構成了開運算，閉運算，梯度等。

## 9-3: 侵蝕、膨脹、開運算、閉運算

>### Erode 侵蝕 : 以 Kernel 中心點移動
> 卷積核沿著圖像滑動，如果與卷積核對應的原圖像的`所有圖元值與 Kernel 相同, 那麼中心元素就給 1，否則就變為零`。根據卷積核的大小靠近前景的所有圖元都會被侵蝕掉（變為0），所以`前景物體會變小`，整幅圖像的白色區域會減少。這對於`去除白色雜訊很有用`，也可以用來斷開兩個連在一塊的物體等。  

> cv2.erode(img, kernel=None, iterations =1)

> * img : 指需要侵蝕的圖
> * kernel : 指侵蝕操作的內核，默認是一個簡單的 3X3 全 1 的矩陣，我們也可以利用 getStructuringElement（）函數指明它的形狀
> * iterations : 指的是侵蝕次數，省略是默認為1

>在進行侵蝕和膨脹的講解之前，首先需要注意，侵蝕和膨脹是對白色部分（高亮部分）而言的，不是黑色部分。膨脹就是影像中的高亮部分進行膨脹，`領域擴張`，效果圖擁有比原圖更大的高亮區域。侵蝕就是原圖中的高亮部分被侵蝕，`領域被蠶食`，效果圖擁有比原圖更小的高亮區域

>><img src="../assets/images/basic/erode.jpg"  style='width:100%'></img>

In [ ]:
import cv2
import numpy as np
img=np.zeros((7, 7), np.uint8)
img[1:6, 1:6]=1

kernel = np.ones((3, 3),np.uint8)  # try  [3, 3]
erosion = cv2.erode(img, kernel, iterations = 1)     # 調整 interation 試一試
print(f'img =\n{img}\n\n'
      f'kernel =\n{kernel}\n\n'
      f'erosion =\n{erosion}')

In [ ]:
import cv2
import numpy as np
o=cv2.imread('../assets/images/basic/erode.bmp')
kernel33 = np.ones((3, 3), np.uint8)
erosion33 = cv2.erode(o, kernel33, iterations = 1)     # 調整 interation 試一試

kernel55 = np.ones((5,5), np.uint8)
erosion55 = cv2.erode(o, kernel55, iterations = 1)     # 調整 interation 試一試

cv2.imshow('original', o)
cv2.imshow('erosion33', erosion33)
cv2.imshow('erosion55', erosion55)
cv2.waitKey()
cv2.destroyAllWindows()
cv2.waitKey(1)

### Dilate 膨脹
> 與侵蝕相反，與卷積核對應的原圖像的圖元值中`只要有一個是 1`，中心元素的圖元值就是 1。所以這個操作會增加圖像中的白色區域（前景）。一般在去雜訊時先用侵蝕再用膨脹。因為侵蝕在去掉白色雜訊的同時，也會使前景對像變小。所以我們再對他進行膨脹。這時雜訊已經被去除了，不會再回來了，但是前景還在並會增加。

> 膨脹也可以用來連接兩個分開的物體。其實，膨脹就是求區域性最大值的操作。按數學方面來說，膨脹或者侵蝕操作就是將影像（或影像的一部分割槽域，我們稱之為 A）與核（我們稱之為 B）進行卷積。

>核可以是任何的形狀和大小，它擁有一個單獨定義出來的參考點，我們稱其為錨點（anchorpoint）。多數情況下，核是一個小的中間帶有參考點和實心正方形或者圓盤，其實，我們可以把核視為模板或者掩碼。

>而膨脹就是求區域性最大值的操作，核B與圖形卷積，即計算核B覆蓋的區域的畫素點的最大值，並把這個最大值賦值給參考點指定的畫素。這樣就會使影像中的高亮區域逐漸增長。如下圖所示，這就是膨脹操作的初衷。<br>
>> <img src="../assets/images/basic/dilate.jpg"  style='width:100%'></img>

In [ ]:
import cv2
import numpy as np

img=np.zeros((5, 5),np.uint8)
img[2:3, 1:4]=1

kernel = np.ones((3,1), np.uint8)
dilation = cv2.dilate(img, kernel)

print(f'img=\n{img}\n\n'
      f'kernel=\n{kernel}\n\n'
      f'dilation\n{dilation}')

### 不同大小的 Kernel 膨脹 dilate

In [ ]:
import cv2
import numpy as np
o=cv2.imread('../assets/images/basic/dilation.bmp')

kernel55 = np.ones((5, 5), np.uint8)
kernel99 = np.ones((9 ,9), np.uint8)

dilation55 = cv2.dilate(o, kernel55)
dilation99 = cv2.dilate(o, kernel99)

cv2.imshow('original',o)
cv2.imshow('dilation55', dilation55)
cv2.imshow('dilation99', dilation99)

cv2.waitKey()
cv2.destroyAllWindows()
cv2.waitKey(1)

### erode & dilate

In [ ]:
import numpy as np
import argparse
import cv2

image = cv2.imread('../assets/images/basic/coins.jpg')
cv2.imshow('Original', image)

gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
# cv2.imshow('Gray', gray)

th, thresh = cv2.threshold(gray, 225, 255, cv2.THRESH_BINARY_INV)  # 先二值 0, 255
cv2.imshow(f'Thresh : {th}', thresh)

# we apply erosions to reduce the size of foreground objects
mask = cv2.erode(thresh.copy(), None, iterations=1)    # None : kernel  default is a simple 3x3 matrix
cv2.imshow('Eroded', mask)

# similarly, dilations can increase the size of the ground objects
mask = cv2.dilate(thresh.copy(), None, iterations=1)   # None : kernel  default is a simple 3x3 matrix
cv2.imshow('Dilated', mask)

cv2.waitKey(0)
cv2.destroyAllWindows()
cv2.waitKey(1)

### Open : MORPH_OPEN 
> 先 erode 再 dilate 叫 open 運算，作用能消除圖片上的小標點<br>
> 降噪, 計數

In [ ]:
import cv2
import numpy as np
img1=cv2.imread('../assets/images/basic/opening.bmp')
img2=cv2.imread('../assets/images/basic/opening2.bmp')

k=np.ones((10,10),np.uint8)

r1=cv2.morphologyEx(img1, cv2.MORPH_OPEN, k)   # MORPH_OPEN
r2=cv2.morphologyEx(img2, cv2.MORPH_OPEN, k)

cv2.imshow('img1', img1)
cv2.imshow('result1', r1)
cv2.imshow('img2', img2)
cv2.imshow('result2', r2)

cv2.waitKey()
cv2.destroyAllWindows()
cv2.waitKey(1)

### Close : MORPH_CLOSE
> 先 dilate, 再 erode : 它經常被用來填充前景物體中的小洞，或者前景物體上的小黑點。不同前景影像連接 

In [ ]:
import cv2
import numpy as np
img=cv2.imread('../assets/images/basic/closing.bmp')

k=np.ones((11, 11),np.uint8)

r=cv2.morphologyEx(img, cv2.MORPH_CLOSE, k, iterations=1)

cv2.imshow('img1',img)
cv2.imshow('result1',r)

cv2.waitKey()
cv2.destroyAllWindows()
cv2.waitKey(1)

### opening vs. closing

In [ ]:
import cv2
import numpy as np

img1 = cv2.imread('../assets/images/basic/opening.png', 0)
img2 = cv2.imread('../assets/images/basic/closing.png', 0)

kernel = np.ones((5, 5), np.uint8)

opening = cv2.morphologyEx(img1, cv2.MORPH_OPEN, kernel)
cv2.imshow('img1', cv2.resize(img1, (360, 240)))
cv2.imshow('img1 Opening', cv2.resize(opening, (360, 240)))

closing = cv2.morphologyEx(img2, cv2.MORPH_CLOSE, kernel)
cv2.imshow('img2', cv2.resize(img2, (360, 240)))
cv2.imshow('img2 Closing', cv2.resize(closing, (360, 240)))

cv2.waitKey(0)
cv2.destroyAllWindows()
cv2.waitKey(1)

### Gradient 梯度  (膨脹 - 侵蝕)
> 取得前景原始影像的邊緣<br>
> 用於獲取圖片的輪廓，形態梯度圖 = 膨脹圖 - 侵蝕圖

In [ ]:
import cv2
import numpy as np
o=cv2.imread('../assets/images/basic/gradient.bmp')
k=np.ones((5, 5), np.uint8)

d = cv2.dilate(o, k)  # try to add iteration
e = cv2.erode(o, k)

r=cv2.morphologyEx(o, cv2.MORPH_GRADIENT, k)

cv2.imshow('original', o)
# cv2.imshow('erode', e)
# cv2.imshow('deilate', d)
cv2.imshow('Gradient', r)
cv2.imshow('d-e', d-e)
cv2.imshow('d-o', d-o)

cv2.waitKey()
cv2.destroyAllWindows()
cv2.waitKey(1)

### Tophat 禮帽 ( 原圖 - Open )
> 取得原影像雜訊資訊<br>
> 取得比原影像邊緣更 `亮` 的邊緣<br>
> Top Hat = 原圖 -  open 運算圖，顯示出原圖去除掉的白色部分。

In [ ]:
import cv2
import numpy as np
o=cv2.imread('../assets/images/basic/tophat.bmp')

k=np.ones((5,5),np.uint8)

r=cv2.morphologyEx(o, cv2.MORPH_TOPHAT, k)

cv2.imshow('original',o)
cv2.imshow('result',r)

cv2.waitKey()
cv2.destroyAllWindows()
cv2.waitKey(1)

### Blackhat( close - 原圖 )
> 取得原影內部小孔, 或前景中的小黑點<br>
> 取得比原影像邊緣更 `暗` 的邊緣<br>
> Black Hat = 原圖 - 閉運算， 顯示出原圖去除掉的黑色部分。

In [ ]:
import cv2
import numpy as np
o=cv2.imread('../assets/images/basic/blackhat.bmp')
k=np.ones((9,9), np.uint8)

r=cv2.morphologyEx(o, cv2.MORPH_BLACKHAT, k)

cv2.imshow('original',o)
cv2.imshow('result', r)

cv2.waitKey()
cv2.destroyAllWindows()
cv2.waitKey(1)

### Define Own Kernel
* MORPH_RECT : 矩形結構，所有元素是 1
* MORPH_CROSS ：十字結構, 對角線元素是 1
* MORPH_ELLIPSE : 橢圓形結構，所有元素是 1

In [ ]:
import cv2
import numpy as np

k1 = cv2.getStructuringElement(cv2.MORPH_RECT, (15, 15))  # same np.ones([5, 5])
k2 = cv2.getStructuringElement(cv2.MORPH_CROSS, (15, 15))
k3 = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (15, 15))

print(f'kernel1=\n{k1}\n\n'
      f'kernel2=\n{k2}\n\n'
      f'kernel3=\n{k3}')

In [ ]:
# import cv2
import numpy as np

o = cv2.imread('../assets/images/basic/kernel.bmp')

k1 = cv2.getStructuringElement(cv2.MORPH_RECT, (50,50))
k2 = cv2.getStructuringElement(cv2.MORPH_CROSS, (50,50))
k3 = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (50,50))

d1 = cv2.dilate(o, k1)
d2 = cv2.dilate(o, k2)
d3 = cv2.dilate(o, k3)

cv2.imshow('original', o)
cv2.imshow('d1 rect', d1)
cv2.imshow('d2 cross', d2)
cv2.imshow('d3 ellipse', d3)

cv2.waitKey()
cv2.destroyAllWindows()
cv2.waitKey(1)

In [ ]:
import cv2
import numpy as np
from matplotlib import pyplot as plt

img = cv2.imread('../assets/images/basic/SpongeBob.jpg')
img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

kernel = np.ones((3, 3), np.uint8)  # 卷積核

erosion = cv2.erode(img, kernel, iterations=1)  # 腐蝕
dilation = cv2.dilate(img, kernel, iterations=1)  # 膨脹
opening = cv2.morphologyEx(img, cv2.MORPH_OPEN, kernel)  # 開運算
closing = cv2.morphologyEx(img, cv2.MORPH_CLOSE, kernel)  # 閉運算
gradient = cv2.morphologyEx(img, cv2.MORPH_GRADIENT, kernel)  # 形態學梯度
tophat = cv2.morphologyEx(img, cv2.MORPH_TOPHAT, kernel)  # 禮帽
blackhat = cv2.morphologyEx(img, cv2.MORPH_BLACKHAT, kernel)  # 黑帽

plt.figure(figsize=(12, 18))
plt.subplot(421), plt.imshow(img), plt.title('Original')
plt.subplot(422), plt.imshow(erosion), plt.title('Erosion')
plt.subplot(423), plt.imshow(dilation), plt.title('Dilation')
plt.subplot(424), plt.imshow(opening), plt.title('Opening')
plt.subplot(425), plt.imshow(closing), plt.title('Closing')
plt.subplot(426), plt.imshow(gradient), plt.title('Gradient')
plt.subplot(427), plt.imshow(tophat), plt.title('Tophat')
plt.subplot(428), plt.imshow(blackhat), plt.title('Blackhat')

plt.show()

<div style="page-break-after: always"></div>

### Summary : 
| 中文名    | 英文名 | api | 原理 | 個人理解 |
|:--------:|:------:|:---:|:----:|:--------:|
|  侵蝕    | erode             | erosion = cv2.erode(src=girl_pic, kernel=kernel) | 在窗中，只要含有０，則窗內全變為０，可以去淺色噪點	| 淺色成分被侵蝕 |
|  膨脹    | dilate            | dilation = cv2.dilate(src=girl_pic, kernel=kernel) | 在窗中，只要含有１，則窗內全變為１，可以增加淺色成分 | 淺色成分得膨脹 |
| 開運算   | morphology-open   | opening = cv2.morphologyEx(girl_pic, cv2.MORPH_OPEN, kernel) | 先侵蝕，後膨脹，去白噪點 | 先合再開，對淺色成分不利 |
| 閉運算   | morphology-close  | closing = cv2.morphologyEx(girl_pic, cv2.MORPH_CLOSE, kernel) | 先膨脹，後侵蝕，去黑噪點 | 先開再合，淺色成分得勢 |
|形態學梯度|morphology-grandient|gradient = cv2.morphologyEx(girl_pic, cv2.MORPH_GRADIENT, kernel)|一幅影像侵蝕與膨脹的區別，可以得到輪廓|數值上解釋為：膨脹減去侵蝕|
| 禮帽     | tophat             |	tophat = cv2.morphologyEx(girl_pic, cv2.MORPH_TOPHAT, kernel) |	原影像減去開運算的差 | 數值上解釋為：原影像減去開運算 |
| 黑帽     | blackhat           | blackhat = cv2.morphologyEx(girl_pic, cv2.MORPH_BLACKHAT, kernel) | 閉運算減去原影像的差 | 數值上解釋為：閉運算減去原影像 |

---